In [1]:
! pip install emcee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.1 MB/s eta 0:00:00


In [6]:
! git clone https://github.com/fsorrenti/debug.git

Cloning into 'debug'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 2), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 10.07 MiB | 15.82 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [11]:
%cd debug

/content/debug


In [24]:
from __future__ import annotations
import sys
import numpy as np
from scipy.stats import multivariate_normal
import dipole_utilities as ut
import emcee


ModuleNotFoundError: No module named 'dipole_utilities'

In [25]:
from astropy.coordinates import SkyCoord, spherical_to_cartesian
from astropy import units as u
import math as m
from scipy import integrate

In [26]:
covf = open("Pantheon+SH0ES_STAT+SYS.cov").readlines()
covl = np.array(covf, dtype="float")
shape = covl[0]  # si prende la misura in questo modo, geniale
covmat = covl[1:].reshape((int(shape), int(shape)))

inv=np.linalg.inv(covmat)


filtered_data_file = np.genfromtxt(
    "Pantheon+SH0ES.dat", usecols=(4, 10, 12, 13, 26, 27, 31, 8), skip_header=1
)


z = filtered_data_file[:, 0]
mu = filtered_data_file[:, 1]
ceph_dis = filtered_data_file[:, 2]
is_cal = filtered_data_file[:, 3]
ra = filtered_data_file[:, 4]
dec = filtered_data_file[:, 5]
v_pec = filtered_data_file[:, 6]
m_b = filtered_data_file[:, 7]


In [27]:
def integrand(z, H0, Omat):
    return 1 / (H0 * m.sqrt(Omat * (1 + z) ** 3 + (1 - Omat)))


def dl_monopole(z, H0, Omat):
    c = 299792.458
    return (
        c * (1 + z) * integrate.romberg(integrand, 0, z, args=(H0, Omat))
    )  # return quad(integrand, 0, z, args=(H0, Omat, Olam))[0]

In [28]:
def monopole(z, H0, Omat):
    return 5 * np.log10((dl_monopole(z, H0, Omat))) + 25

In [29]:
def exp_obs_monopole_with_M(
    M, H0, omega_m, mu, z, is_cal, ceph,
):
    number_of_elements = len(mu)
    exp_obs_mon = np.zeros(number_of_elements)

    for i in range(0, number_of_elements):
        if is_cal[i] == 1:
            exp_obs_mon[i] = mu[i] + M - ceph[i]
            # exp_obs_mon[i]=mu[i]-monopole(z6[i],H0,omega_m)
        else:
            exp_obs_mon[i] = mu[i] + M - monopole(z[i], H0, omega_m)

    return exp_obs_mon

In [30]:
def log_likelihood_agnostic(parameters: list[float], inversed_covariance):

    '''the following for the agnostic taylor expansion'''
#    d1, d2, d3, dM  = parameters

#    res=residual_agnostic_taylor_expansion(d1, d2, d3, dM )

  #  z0, d1, d2, d3, dM  = parameters

    Omat,dM  = parameters



    res=exp_obs_monopole_with_M(dM,73.4, Omat, mu, z, is_cal, ceph_dis)


    chi2 = res @ inversed_covariance @ res
    return -0.5 * chi2  # normalization is constant so can be omitted


In [31]:
def log_prior(theta, priors):

    """
    log-prior for the parameters (uniform)
    """
    # theta is the free parameters, priors is a list of two elements
    mu, halfwidth = priors
    for mu_i, theta_i, halfwidth_i in zip(mu, theta, halfwidth):
        if theta_i > mu_i + halfwidth_i or theta_i < mu_i - halfwidth_i:
            return -np.inf
    return 0


In [32]:
def log_probability(parameters: list[float], priors, inversed_covariance):
    """
    log-probability that governs the rate of acceptance for given proposed parameter.
    """
    lp = log_prior(parameters, priors)
    if not np.isfinite(lp):
        return -np.inf

    ll = log_likelihood_agnostic(parameters, inversed_covariance)
    if not np.isfinite(ll):
        return -np.inf

    return lp + ll

In [33]:
def run_MCMC(
    nwalkers,
    number_of_steps,
    parameters_values,
    prior_width,
    outpath=None,
    contd=None,
):
    """
    a wrapper file
    """

    inversed_covariance = inv

    # inversed_covariance=inversed_covariance_data



    if outpath is None:
        outpath = "./results.h5"
    # load data

    # priors
    reference_values = np.array(parameters_values, dtype=float)
    widths = np.array(prior_width)
    priors = [reference_values, widths]

    # initial guess array for each walker:
    x0 = np.random.uniform(
        reference_values - widths,
        reference_values + widths,
        size=(nwalkers, len(reference_values)),
    )
    nwalkers, ndim = x0.shape

    # save file
    backend = emcee.backends.HDFBackend(outpath)
    if contd:  # it is true if you want to continue the run
        print(f"initial size: {backend.iteration}", flush=True)
    else:
        backend.reset(nwalkers, ndim)

        # initialize sampler, run MCMC
    sampler = emcee.EnsembleSampler(
        nwalkers,
        ndim,
        log_probability,
        args=[priors, inversed_covariance],
        backend=backend,
    )

    if contd:
        sampler.run_mcmc(
            None, number_of_steps, progress=True, skip_initial_state_check=True
        )
        print(f"final size: {backend.iteration}", flush=True)
    else:
        sampler.run_mcmc(
            x0, number_of_steps, progress=True, skip_initial_state_check=True
        )


In [34]:
# default output file name
OUTPATH = "test_scratch.h5"

# MCMC sampler settings
N_WALKERS = 32  # number of walkers
N_CHAIN = 1000  # length of chain for each walker
CONTD = False  # set CONTD = True to resume sampling (for specific OUTPATH)
# note: keeping CONTD = False may overwrite an existing file.

PARAMETERS_VALUES = [0.5, 0]#per la mia scelta di angoli il coseno puo' essere solo fra 0 e 1
PRIOR_WIDTH = [0.5, 0.5]


In [35]:
if __name__ == "__main__":  # it just execute the function
    # load least-square, bets-fit results by default
    #
    # note:
    # Passing manually defined arrays to 'lstsq_results' argument
    # and setting PRIOR_WIDTH_RATIO = 0 will allow users to define
    # the desired central values and width of the prior manually.
    # If you wish to use different (e.g., Gaussian) priors, define
    # the analytic Gaussian function in MCMC_utils.py file accordingly.
    # q_lstsq, sigma_lstsq = np.loadtxt(lstsq_results_path, unpack=True)

    # run MCMC
    run_MCMC(
        nwalkers=N_WALKERS,
        number_of_steps=N_CHAIN,
        parameters_values=PARAMETERS_VALUES,
        prior_width=PRIOR_WIDTH,
        outpath=OUTPATH,
        contd=CONTD,
    )


<ipython-input-27-6fc222fc8025>:8: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  c * (1 + z) * integrate.romberg(integrand, 0, z, args=(H0, Omat))
  0%|          | 73/1000000 [05:52<1459:35:34,  5.25s/it]Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/emcee/ensemble.py", line 640, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-32-817bfee58354>", line 9, in log_probability
    ll = log_likelihood_agnostic(parameters, inversed_covariance)
  File "<ipython-input-30-ee57b9ec58b3>", line 14, in log_likelihood_agnostic
    res=exp_obs_monopole_with_M(dM,73.4, Omat, mu, z, is_cal, ceph_dis)
  File "<ipython-input-29-42f890cadbf2>", line 12, in exp_obs_monopole_with_M
    exp_obs_mon[i] = mu[i] + M - monopole(z[i], H0, omega_m)
  File "<ipython-input-28-f983de8cdca2>", line 2, in monopole
    return 5 * np.log10(

emcee: Exception while calling your likelihood function:
  params: [ 0.34257344 -0.0170606 ]
  args: [[array([0.5, 0. ]), array([0.5, 0.5])], array([[ 3.35405114e+01, -5.68645204e+00, -1.07870284e-01, ...,
         1.44562418e-02,  1.73619146e-02,  6.56463541e-02],
       [-5.68645201e+00,  3.26679150e+01,  6.27809019e-01, ...,
         5.90524514e-02,  5.86751704e-02,  9.18237282e-02],
       [-1.07870328e-01,  6.27808965e-01,  5.66302667e+01, ...,
         4.04987268e-02,  6.08045796e-02,  1.44339360e-01],
       ...,
       [ 1.44562147e-02,  5.90524254e-02,  4.04987139e-02, ...,
         2.64804296e+01, -2.56390503e-01, -5.18344051e-01],
       [ 1.73618976e-02,  5.86751545e-02,  6.08045714e-02, ...,
        -2.56390503e-01,  1.63294720e+01, -4.65796138e-01],
       [ 6.56463083e-02,  9.18236857e-02,  1.44339335e-01, ...,
        -5.18344052e-01, -4.65796138e-01,  2.87369713e+01]])]
  kwargs: {}
  exception:


KeyboardInterrupt: 